In [1]:
from pathlib import Path
import sys
import cobra
import os
import pandas as pd
import re

In [2]:
path_root = Path("C:/Users/tobyl/OneDrive - The University of Manchester/Bioinformatics Masters/Research Project 2/Pycomo/PyCoMo/src")
sys.path.append(str(path_root))
import pycomo


### Creating community model
1. Load the member models

In [3]:
# loading member models
initial_path = Path().absolute().parent

model_member_dir = initial_path / r"old_files/models_members_test/Toy"
named_models = pycomo.load_named_models_from_dir(model_member_dir)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2534040
Academic license 2534040 - for non-commercial use only - registered to to___@postgrad.manchester.ac.uk


In [4]:
# checking modesl that have been loaded
print(named_models)

{'Bacteroides': <Model Bacteroides_uniformis_ATCC_8492 at 0x2290d1e6390>, 'Bifidobacterium': <Model Bifidobacterium_adolescentis_ATCC_15703 at 0x2291713c350>, 'Faecalibacterium': <Model Faecalibacterium_prausnitzii_M21_2 at 0x229193b55e0>}


In [ ]:
# checking if I can edit the models
model1 = named_models['Bacteroides']
for reaction in model1.exchanges:
    print(reaction.name)
    print(reaction)
    print(reaction.bounds)

2. checking the objective function of the loaded models

In [5]:
# if all have a biomass objective function then it is working
for model in named_models.values():
    print(model.objective)

Maximize
1.0*biomass536 - 1.0*biomass536_reverse_50d4e
Maximize
1.0*biomass345 - 1.0*biomass345_reverse_e128f
Maximize
1.0*biomass224 - 1.0*biomass224_reverse_c48a3


In [6]:
# creating the single organism models in PyCoMo format
single_org_models = []
for name, model in named_models.items():
    print(name)
    single_org_model = pycomo.SingleOrganismModel(model, name)
    single_org_models.append(single_org_model)

Bacteroides
Read LP format model from file C:\Users\tobyl\AppData\Local\Temp\tmp3zcdu4q9.lp
Reading time = 0.03 seconds
: 1500 rows, 4836 columns, 17482 nonzeros
Bifidobacterium
Read LP format model from file C:\Users\tobyl\AppData\Local\Temp\tmphq5927bt.lp
Reading time = 0.01 seconds
: 991 rows, 2258 columns, 9484 nonzeros
Faecalibacterium
Read LP format model from file C:\Users\tobyl\AppData\Local\Temp\tmp6bz6hdbq.lp
Reading time = 0.02 seconds
: 1361 rows, 3534 columns, 13158 nonzeros


3. Creating the community model

In [7]:
community_name = "ten_gutControl_model"
com_model_obj = pycomo.CommunityModel(single_org_models, community_name)

In [11]:
##### medium creation code
import re

parent_path = Path().absolute().parent
diet_filepath = parent_path / r"diet_info/western_fluxes.tsv"
diet_df = pd.read_csv(diet_filepath, sep = "\t")
diet_df.loc[:, 'Reaction'] = diet_df['Reaction'].apply(lambda reaction: re.split(r"\[|\(", reaction)[0])
diet_reactionID_list = list(diet_df.loc[:, 'Reaction'])


#creating the medium with the diet values
medium_exchange_dict = {}
for medium_reaction in com_model_obj.model.exchanges:
    medium_reaction_id = medium_reaction.id

    # ensuring medium_reaction_id matches ids from diet_reaction_ID_list
    medium_reaction_parts = medium_reaction_id.split("_")
    if medium_reaction_parts[1] == "":
        del(medium_reaction_parts)[1]
    del(medium_reaction_parts)[-1]
    reaction_id = "_".join(medium_reaction_parts)

    if reaction_id in medium_exchange_dict.values():
        continue
    elif reaction_id in diet_reactionID_list:
        flux = diet_df.loc[diet_df['Reaction'] == reaction_id, 'Flux Value'].values
        medium_exchange_dict[medium_reaction_id] = float(flux[0])
    else:
            medium_exchange_dict[medium_reaction_id] = 1e-6
print(medium_exchange_dict)

        
           
           
       

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
{'EX__12dgr180_medium': 1e-06, 'EX__3mop_medium': 1e-06, 'EX_Lcyst_medium': 1e-06, 'EX_Lcystin_medium': 1e-06, 'EX_MGlcn100_medium': 1e-06, 'EX_MGlcn100_rl_medium': 1e-06, 'EX_MGlcn101_medium': 1e-06, 'EX_MGlcn101_rl_medium': 1e-06, 'EX_MGlcn102_medium': 1e-06, 'EX_MGlcn102_rl_medium': 1e-06, 'EX_MGlcn104_medium': 1e-06, 'EX_MGlcn104_rl_medium': 1e-06, 'EX_MGlcn105_medium': 1e-06, 'EX_MGlcn105_rl_medium': 1e-06, 'EX_MGlcn106_medium': 1e-06, 'EX_MGlcn106_rl_medium': 1e-06, 'EX_MGlcn107_medium': 1e-06, 'EX_MGlcn107_rl_medium': 1e-06, 'EX_MGlcn109_medium': 1e-06, 'EX_MGlcn109_rl_medium': 1e-06, 'EX_MGlcn110_medium': 1e-06, 'EX_MGlcn110_rl_medium': 1e-06, 'EX_MGlcn111_medium': 1e-06, 'EX_MGlcn111_rl_medium': 1e-06, 'EX_MGlcn112_medium': 1e-06, 'EX_MGlcn112_rl_medi

In [8]:
# calling the community model to make it
com_model_obj.model

No community model generated yet. Generating now:
Read LP format model from file C:\Users\tobyl\AppData\Local\Temp\tmpt2rdkzep.lp
Reading time = 0.03 seconds
: 1500 rows, 4836 columns, 17482 nonzeros
Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Read LP format model from file C:\Users\tobyl\AppData\Local\Temp\tmpu6kmmd4i.lp
Reading time = 0.01 seconds
: 991 rows, 2258 columns, 9484 nonzeros
Note: no products in the objective function, adding biomass to it.


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Read LP format model from file C:\Users\tobyl\AppData\Local\Temp\tmpv548uq7l.lp
Reading time = 0.05 seconds
: 2754 rows, 5786 columns, 19766 nonzeros


...

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\tobyl\miniconda3\envs\masterswork\Lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, in

Read LP format model from file C:\Users\tobyl\AppData\Local\Temp\tmp5euw3xng.lp
Reading time = 0.07 seconds
: 4376 rows, 9562 columns, 30400 nonzeros


Ignoring reaction 'EX_MGlcn100_medium' since it already exists.
Ignoring reaction 'EX_MGlcn100_rl_medium' since it already exists.
Ignoring reaction 'EX_MGlcn101_medium' since it already exists.
Ignoring reaction 'EX_MGlcn101_rl_medium' since it already exists.
Ignoring reaction 'EX_MGlcn102_medium' since it already exists.
Ignoring reaction 'EX_MGlcn102_rl_medium' since it already exists.
Ignoring reaction 'EX_MGlcn104_medium' since it already exists.
Ignoring reaction 'EX_MGlcn104_rl_medium' since it already exists.
Ignoring reaction 'EX_MGlcn105_medium' since it already exists.
Ignoring reaction 'EX_MGlcn105_rl_medium' since it already exists.
Ignoring reaction 'EX_MGlcn106_medium' since it already exists.
Ignoring reaction 'EX_MGlcn106_rl_medium' since it already exists.
Ignoring reaction 'EX_MGlcn107_medium' since it already exists.
Ignoring reaction 'EX_MGlcn107_rl_medium' since it already exists.
Ignoring reaction 'EX_MGlcn109_medium' since it already exists.
Ignoring reaction '

Generated community model.


Name,ten_gutControl_model
Memory address,22919dc31a0
Number of metabolites,12062
Number of reactions,13527
Number of genes,2164
Number of groups,74
Objective expression,1.0*community_biomass - 1.0*community_biomass_reverse_44dc1
Compartments,"Bacteroides_c, Bacteroides_e, Bacteroides_medium, medium, fraction_reaction, Bifidobacterium_e, Bifidobacterium_c, Bifidobacterium_medium, Faecalibacterium_e, Faecalibacterium_c, Faecalibacterium_medium"


In [ ]:
##### medium creation code


model_members = com_model_obj.member_models
single_org_model = model_members[0]
model = single_org_model.model
for reaction in model.exchanges:
    print(reaction.id)
    print(reaction)
    print(reaction.bounds)

### Getting the Summary and Report for the model

In [9]:
com_model_obj.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
_4abz_medium,EX__4abz_medium,0.00929,7,0.05%
asn_L_medium,EX_asn_L_medium,0.2008,4,0.67%
asp_L_medium,EX_asp_L_medium,4.442,4,14.74%
ca2_medium,EX_ca2_medium,0.003096,0,0.00%
cl_medium,EX_cl_medium,0.003096,0,0.00%
cobalt2_medium,EX_cobalt2_medium,0.003096,0,0.00%
cu2_medium,EX_cu2_medium,0.003096,0,0.00%
cys_L_medium,EX_cys_L_medium,0.07615,3,0.19%
ddca_medium,EX_ddca_medium,0.2352,12,2.34%
etoh_medium,EX_etoh_medium,1.825,2,3.03%


- In report, need to check any unbalanced reactions that are not sink/demand (SINK, DM), Exchange (EX), Transport (T_ or TR_) or BIOMASS

In [ ]:
com_model_obj.report()

In [17]:
for reaction in com_model_obj.f_reactions:
    if "biomass" in reaction.id:
        print(reaction.id)
        try:
            reaction.flux
        except:
            print("oh dear")

SK_Bacteroides_EX_biomass_e_Bacteroides_c_lb
oh dear
SK_Bacteroides_EX_biomass_e_Bacteroides_c_ub
oh dear
SK_Bacteroides_biomass536_ub
oh dear
SK_Bacteroides_to_community_biomass_ub
oh dear
SK_Bifidobacterium_EX_biomass_e_Bifidobacterium_c_lb
oh dear
SK_Bifidobacterium_EX_biomass_e_Bifidobacterium_c_ub
oh dear
SK_Bifidobacterium_biomass345_ub
oh dear
SK_Bifidobacterium_to_community_biomass_ub
oh dear
SK_Faecalibacterium_EX_biomass_e_Faecalibacterium_c_lb
oh dear
SK_Faecalibacterium_EX_biomass_e_Faecalibacterium_c_ub
oh dear
SK_Faecalibacterium_biomass224_ub
oh dear
SK_Faecalibacterium_to_community_biomass_ub
oh dear


- Checking if there are any thermodynamically infeasible cycles

In [ ]:
# com_model_obj.get_loops()
# taking too long to run so I stopped it
# could be a problem and will have to 

## Step 3: Converting to fixed abundances as we are modelling specific conditions

In [10]:
# creating the abundance_dict
abundance_dict = com_model_obj.generate_equal_abundance_dict()
print(sum(abundance_dict.values()))
print(abundance_dict)

1.0
{'Bacteroides': 0.3333333333333333, 'Bifidobacterium': 0.3333333333333333, 'Faecalibacterium': 0.3333333333333333}


In [11]:
# converting to fixed abundance model and then adding the abundances from abundance_dict
com_model_obj.convert_to_fixed_abundance()
com_model_obj.apply_fixed_abundance(abundance_dict)

# summarising the model
com_model_obj.summary()





Metabolite,Reaction,Flux,C-Number,C-Flux
Lcystin_medium,EX_Lcystin_medium,2.564,6,0.07%
_12dgr180_medium,EX__12dgr180_medium,0.1461,39,0.03%
_26dap_M_medium,EX__26dap_M_medium,0.5881,7,0.02%
_4abz_medium,EX__4abz_medium,0.1878,7,0.01%
acald_medium,EX_acald_medium,333.3,2,3.25%
alagln_medium,EX_alagln_medium,301.7,8,11.76%
alathr_medium,EX_alathr_medium,3.778,7,0.13%
amp_medium,EX_amp_medium,25.11,10,1.22%
arg_L_medium,EX_arg_L_medium,5.829,6,0.17%
asn_L_medium,EX_asn_L_medium,38.16,4,0.74%


In [14]:
solution = com_model_obj.model.optimize()
for reaction_name, flux in solution.fluxes.items():
    if "to_community_biomass" in reaction_name and "SK" not in reaction_name:
        print(reaction_name)
        print(flux)

Bacteroides_to_community_biomass
20.219140374057336
Bifidobacterium_to_community_biomass
20.219140374057336
Faecalibacterium_to_community_biomass
20.219140374057336


Calculating the meatbolic exchanges in FBA. If there are any where the species can loop from consumer to producer there may be a problem

In [15]:
# xechnages under the FBA conditions
exchanges = com_model_obj.potential_metabolite_exchanges(fba= True)
print(exchanges)

          metabolite_id                      metabolite_name  cross_feeding  \
0       _26dap_M_medium        meso-2,6-Diaminoheptanedioate          False   
1         _4abut_medium                     4-Aminobutanoate          False   
2         C02528_medium                    chenodeoxycholate          False   
3       MGlcn100_medium           mucin-type O-glycan No 100          False   
4    MGlcn100_rl_medium  released mucin-type O-glycan No 100          False   
..                  ...                                  ...            ...   
617           q8_medium                         Ubiquinone-8          False   
618      rbflvrd_medium                   Reduced riboflavin          False   
619       sulfac_medium                         sulfoacetate          False   
620          ura_medium                               uracil          False   
621     cpd11416_medium                    Community Biomass          False   

           produced_by         consumed_by  
0     

In [ ]:
summary = com_model_obj.summary()

up_flux = summary.uptake_flux # how to get the uptake flux
sec_flux = summary.secretion_flux # how to get the secretion flux

# can get the objective value and status from this!
solution = com_model_obj.model.optimize()
print(solution.objective_value) # how to get the objective_value
print(solution.status) # How to get the status

print(up_flux.head())
print(sec_flux.head())


- Now, we can check all the potential exchange metabolic fluxes and cross-feeding interactions

In [ ]:
MES = com_model_obj.potential_metabolite_exchanges()

In [ ]:
MES.to_csv("mes_results.csv")